# Assignment 03: Simple Language Model

This week, we want you to create a Language Model (LM) to judge how common a sentence is.  
More specificly, you need to <u>calculate the probability of a given setnence</u> showing in an article.  

*No need to make your output exactly the same as ours. As long as it's *reasonable* (which means you can explain it), you get your points.  

## What is a Language Model?

<b>A statistical language model is a probability distribution over sequences of words. Given such a sequence, say of length $m$, it assigns a probability $P(w_1, \ldots ,w_m)$ to the whole sequence.</b>
<i>-- from <a href="https://en.wikipedia.org/wiki/Language_model">Wikipedia</a></i>  

To put it simply, Language Model calculates the probability of a word, a sequence of words, or a sentence.  
This can be useful in many NLP tasks, like machine translation, spelling checking, speech recognition, etc.  

Consider the following two sentence  

> (1) He is looking <font color="green">*to*</font> a new job.  

and 

> (2) He is looking <font color="green">*for*</font> a new job.  


We can see that there's a grammar error in sentence 1, for which the LM should generate a lower probability.  
Hence, if your LM is well-established, you are able to judge which sentence is more correct (or more common) among a set of candidate sentences.  

But how to do so?

## Implementation

### Step 1 - Ngram frequency

First of all, we need to calculate 1- and 2-gram frequency from the corpus.  
**Again**, yes. As we have said in the first class, word/phrase frequency plays an important role in NLP.

You should be familiar with this procedure now, so let's skip the boring explanation.  
<small>*Please refer to assignment 1 and 2 if you have any question.</small>  

In [1]:
import os
from nltk import ngrams
from collections import Counter
with open(os.path.join('data', 'big.txt'), 'r') as f:
  corpus = f.read()

<font color="red">**[ TODO ]**</font> Calculate 1- and 2-gram frequency with the given corpus.

In [2]:
def unigram(text):
    """ [TODO] Get the 1-gram list from string
    @param text: given string
    @return: a list of token
    """
    return text.lower().split()

def bigram(text):
    """ [TODO] Get the 2-gram list """
    text = text.lower()
    tokens = text.split()
    n_grams = ngrams(tokens, 2)
    return [ ' '.join(grams) for grams in n_grams]

# [TODO] calculate the 1-gram frequency of the corpus
uni = unigram(corpus)
uni_freq = Counter(uni)
# [TODO] calculate the 2-gram frequency of the corpus
bi = bigram(corpus)
bi_freq = Counter(bi)

### Step 2: Probability of a word

#### Unigram  
Probability of a word, or a *unigram*, is simple: its occurence devided by the total word count should do the trick.  
$$P(w) = \frac{Freq(w)}{N}$$  
, where $N = $ count of all words.  
But the unigram language model doesn't consider other context words, so we choose to use bigram model here.  

#### Bigram  
As to 2-gram probability, we would use the Maximum Likelihood Estimate (MLE) to calculate it.  

$$P_{mle}(w_i|w_{i-1}) = \frac{Freq(w_{i-1}, w_i)}{Freq(w_{i-1})}$$

For example, if we have a corpus with 

> can you tell me about any good cantonese restaurants  
> tell me about chez panisse  
> can you give me a listing of the kinds of food that are available  

Then the probability of *tell* preceding *me* is $P_{mle}(me|tell) = \frac{2}{2} = 1$.  
Similarly, the probability of *me* preceding *about* is $P_{mle}(about|me) = \frac{2}{3}$.  


<font color="red">**[ TODO ]**</font> Calculate the bigram probability.

In [3]:
def P_mle(word: str, pre_word: str = None):
    """ [TODO] Return the P(word|pre_word), which should be a float """
    text = pre_word+' '+word
    return bi_freq[text]/uni_freq[pre_word]

In [4]:
p1 = P_mle('book', 'read')
p2 = P_mle('books', 'read')
print(f'p1: {p1}; \np2: {p2}')

p1: 0.0; 
p2: 0.005988023952095809


<font color="green">Expected output: </font> `p1` should be **smaller** than `p2` (expetced to be `0`).  

In [5]:
p1 = P_mle('books', 'a')
p2 = P_mle('book', 'a')
print(f'p1: {p1}; \np2: {p2}')

p1: 0.0; 
p2: 0.0004793608521970706


<font color="green">Expected output: </font> `p1` should be **smaller** than `p2` (expetced to be `0`).  

In [6]:
p1 = P_mle('have', 'he')
p2 = P_mle('has', 'he')
print(f'p1: {p1}; \np2: {p2}')

p1: 0.0004998000799680128; 
p2: 0.014594162335065974


<font color="green">Expected output: </font> `p1` should be **smaller** than `p2`.  

### Step 3 - Sentence probability

Now we have the probability of every word and bigram. But how should we combine them and get the probability of a sentence?  

Recall the chain rule in probability: 
$$P(X_1, X_2, X_3, X_4) = P(X_1) \cdot P(X_2|X_1) \cdot P(X_3|X_1,X_2) \cdot P(X_4|X_1, X_2, X_3)$$  
And with Markove Assumption, we know that 
$$P(w_1, w_2, \dots, w_n) \approx P(w_2|w_1) \cdot P(w_3|w_2) \cdot \dots \cdot P(w_n|w_{n-1})$$

For example,
$$P(I\:want\:a\:cake) \approx P(want|I) \cdot P(a|want) \cdot P(cake|a)$$

Since we have already built a method to calculate $P(w_{i}|w_{i-1})$, we can get the probability of a sentence through this equation.  
Note that it's recommended to sum them under $log$-scale to prevent underflow, because gram probabilities are mostly some small floating points, 
$$log(p_1 \cdot p_2 \dots p_n) = log(p_1) + log(p_2) + \dots + log(p_n) $$

<font color="red">**[ TODO ]**</font> Calculate the sentence probability. (Please do so under the `log` scale to prevent underflow)  
<small>*Hint: `math`</small>

In [7]:
unigram('He have to take the medicine.')

['he', 'have', 'to', 'take', 'the', 'medicine.']

In [8]:
from math import log

def sentence_prob(sentence: str, P):
    """ Calculate the probability of a given sentence with given P-model.
    @param sentence: the given sentence
    @param P: the probability model to use. (like your `P_mle` above)
    @return: a float indicating the probability
    """
    words = unigram(sentence)
    prob = 0
    for i in range(0, len(words)-1):
        prob += log(P(words[i+1], words[i]))
        #print(words[i+1] + ' | ' + words[i])
        #print(log(P_mle(words[i+1], words[i])))
    ... # [TODO] calculate the probability

    return prob

In [9]:
p1 = sentence_prob('He have to take the medicine.', P=P_mle)
p2 = sentence_prob('He has to take the medicine.', P=P_mle)
print(f'p1: {p1}; \np2: {p2}')

p1: -29.30948724296647; 
p2: -26.62036562628031


In [10]:
p1 = sentence_prob('He have to take the medicine.', P=P_mle)
p2 = sentence_prob('He has to take the medicine.', P=P_mle)
print(f'p1: {p1}; \np2: {p2}')

p1: -29.30948724296647; 
p2: -26.62036562628031


<font color="green">Expected output: </font> `p1` should be **smaller** than `p2`.  

### Looks good so far?

However, this model heavily depends on how large your dataset is.   
If you give it the word or gram not existing in the corpus, the probability of it will be $0$, which causes problems during the calculation (because you can't divide numbers by $0$).  

Try this! 

In [11]:
sentence_prob("He has to eat medicine.", P=P_mle)

ValueError: math domain error

See? Now you get the problem.  
Surely you can expand your dataset to decrease the number of unseen phrases, but it's impossible to cover all phrases in the world.  
Here the smoothing technique comes to rescue!  

### Step 4 - smoothing

A naïve solution is to set the default frequency as `1`.  
For example, if the original frequency is

> hello: 2  
> world: 1  
> empty: 0  
> null: 0  

, after defaulting the frequency to 1, it would be

> hello: 3  
> world: 2  
> empty: <font color="red">**1**</font><br/>
> null: <font color="red">**1**</font><br/>


And if the frequency is all added by 1, the probability of bigram will now be  
$$ P_{add1} = \frac{Freq(w_{i-1}, w_i)+1}{Freq(w_{i-1})+N}$$
, where $N = count\:of\:all\:tokens$ .  

This is called the **Add-1 Estimation**, or also **Laplace Smoothing**.  

<font color="red">**[ TODO ]**</font> Implement the Laplace smoothing.  

In [12]:
def P_mle(word: str, pre_word: str = None):
    """ [TODO] Return the P(word|pre_word), which should be a float """
    text = pre_word+' '+word
    return bi_freq[text]/uni_freq[pre_word]

In [13]:
def P_add1(word: str, pre_word: str):
    """ [TODO] Return the P(word|pre_word) with Laplace smoothing. """
    text = pre_word+' '+word
    return (bi_freq[text]+1) / (uni_freq[pre_word]+len(uni_freq))

In [14]:
# myself
p1 = P_add1('medicine', 'eat')
p2 = P_add1('medicine', 'take')
print(f'p1: {p1}; \np2: {p2}')

p1: 1.423224172039338e-05; 
p2: 2.8266553600452264e-05


In [15]:
p1 = P_add1('medicine', 'eat')
p2 = P_add1('medicine', 'take')
print(f'p1: {p1}; \np2: {p2}')

p1: 1.423224172039338e-05; 
p2: 2.8266553600452264e-05


<font color="green">Expected output:</font> `p1` shouldn't throw any error, and `p1` should be **smaller** than `p2`. 

As you can see, Add-1 estimation is a really naive method and doesn't always produce good results.  
There are better smooting techiniques like <a href="https://en.wikipedia.org/wiki/Good%E2%80%93Turing_frequency_estimation">Good-Turing Estimation</a> or <a href="https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing">Kneser-Ney Smoothing</a>. Feel free to give it a try!  

## Test your LM

Seems that you've done a great job! Let's give it some tests to see if it works as expected.  

In [16]:
test_cases = [
  'He have to take the medicine.',
  'He has to eat the medicine.',
  'He has to take the medicine.'
]

for sent in test_cases:
  print(sent, sentence_prob(sent, P=P_add1))

He have to take the medicine. -40.5645650850356
He has to eat the medicine. -44.812473116633655
He has to take the medicine. -38.7614966891868


<font color="green">Expected output:</font> The last sentence should return the highest probability. 

In [17]:
test_cases = [
  'He is looking to a job.',
  'He is looking for a job.'
]

for sent in test_cases:
  print(sent, sentence_prob(sent, P=P_add1))

He is looking to a job. -41.658563026081836
He is looking for a job. -40.50484303264153


<font color="green">Expected output:</font> The second sentence should return a higher probability. 

## Think about it
Think about the following questions. **TA will ask your answers during the demo.**  

1. What would happen if you use MLE Estimation to compare two sentences with different lengthes?  
   Examples. (1) *He school nice and happy.* and (2) *He went to school yesterday and had a nice time there.*  

2. According to Q1, Do you think MLE model is fair? If so, why? If not, how could you improve this? (Just think about it; no need to actually implement it)

In [18]:
# dict not have
test_cases = [
  'He is looking for a job but I can not get it.',
  'He is looking for a job.'
]

for sent in test_cases:
  print(sent, sentence_prob(sent, P=P_add1))
print('\n')
test_cases = [
  'He is looking up at the numbers of the houses.',
  'He is looking up.'
]
for sent in test_cases:
  print(sent, sentence_prob(sent, P=P_add1)) # mutli ratio

He is looking for a job but I can not get it. -93.75761559638221
He is looking for a job. -40.50484303264153


He is looking up at the numbers of the houses. -67.49476439206863
He is looking up. -27.053991314917603


**Congratulation!** You've successfully built your own language model.  
You should have caught a glimpse of LM's power despite its plainness.  
Feel free to play around with your LM and improve it, like  
1. using a larger dataset, 
2. applying better smoothing technique, or 
3. considering longer gram dependency (3- to 5-gram are resonable length).  

## TA's note


Remember to <b><a href="https://docs.google.com/spreadsheets/d/1QGeYl5dsD9sFO9SYg4DIKk-xr-yGjRDOOLKZqCLDv2E/edit?usp=sharing">make an appoiment with TA</a> to demo/explain your implementation <u>before <font color="red">10/7 15:30</font></u></b> .  
You should also save your file as {student_id}.ipynb and submit it to <a href="https://eeclass.nthu.edu.tw/course/homework/2384">eeclass</a> .  
**Late submission is not allowed.**  